#### Import Libraries

In [82]:
import psycopg2
import plotly.graph_objects as go
import plotly.subplots as ms
import plotly.express as px
import pandas as pd
import numpy as np
from array import *

#### Establish Connection with Database

In [198]:
connection = psycopg2.connect(database="tickerdb",
                        user='postgres', password='pass123', 
                        host='127.0.0.1', port='5432'
)
  
connection.autocommit = True
cursor = connection.cursor()

#### Fetch data and insert into variables

In [199]:
cursor.execute('''select datetime from ticker;''')
date = cursor.fetchall()
cursor.execute('''select close from ticker;''')
close = cursor.fetchall()
cursor.execute('''select high from ticker;''')
high = cursor.fetchall()
cursor.execute('''select low from ticker;''')
low = cursor.fetchall() 
cursor.execute('''select open from ticker;''')
open = cursor.fetchall()
cursor.execute('''select volume from ticker;''')
volume = cursor.fetchall()
connection.close()

#### Get data ready to pass it to graph function

In [53]:
date_data = [d[0] for d in date]
close_data = [c[0] for c in close]
high_data = [h[0] for h in high]
low_data = [l[0] for l in low]
open_data = [o[0] for o in open]
volume_data = [v[0] for v in volume]

#### Initialize graph with 2 halves and plot it

In [208]:
fig = ms.make_subplots(rows=2,cols=1)

fig.append_trace(go.Candlestick(x=date_data,open=open_data,high=high_data,low=low_data,close=close_data),row=1,col=1)
fig.append_trace(go.Bar(x=date_data,y=volume_data),row=2,col=1)

fig.update(layout_xaxis_rangeslider_visible=False,layout_title = 'HINDALCO',layout_yaxis_title="Stock Price",layout_xaxis_title="Date",layout_plot_bgcolor='#000',layout_height=1000)
    
fig.show()

### SMA Strategy:

#### Calculate two moving averages and plot them on graph:

In [210]:
close = pd.Series(close_data)
date = pd.Series(date_data)

long = close.rolling(window=50,min_periods=1).mean()
short = close.rolling(window=20,min_periods=1).mean()

fig = px.line()
fig.add_trace(go.Scatter(x=date,y=close,name="Close",line=dict(color="cyan")))
fig.add_trace(go.Scatter(x=date,y=long,name="longMA",line=dict(color="red")))
fig.add_trace(go.Scatter(x=date,y=short,name="shortMA",line=dict(color="green")))
fig.update_layout(title="HINDALCO",yaxis_title="Stock Price",xaxis_title="Date")

fig.show()

#### Calculate buy and sell signals

In [190]:
signal = np.where(short > long,1,0)
signal = pd.Series(signal)

position = signal.diff()
buy = np.where(position == 1,close,np.NAN)
sell = np.where(position == -1,close,np.NAN)

#### Plot buy and sell signals on graph

In [212]:
fig = px.line()
fig.add_trace(go.Scatter(x=date,y=close,name="Close Price",line=dict(color="cyan")))
fig.add_trace(go.Scatter(x=date,y=long,name="longMA",line=dict(color="red")))
fig.add_trace(go.Scatter(x=date,y=short,name="shortMA",line=dict(color="green")))
fig.add_trace(go.Scatter(x=date,y=buy,mode="markers",name="buy signal",marker=dict(color="green",size=8,symbol="triangle-up")))
fig.add_trace(go.Scatter(x=date,y=sell,mode="markers",name="sell signal",marker=dict(color="red",size=8,symbol="triangle-down")))
fig.update_layout(title="HINDALCO",yaxis_title="Stock Price",xaxis_title="Date")

fig.show()

#### Calculate and print Total Profit

In [215]:
buy_price = np.where(position == 1,close,0)
total_buy_price= sum(tt_buy,1216)

sell_price = np.where(position == -1,close,0)
total_sell_price = sum(tt_sell,1216)

#Profit = selling price - cost price
total_profit = total_sell_price - total_buy_price
print(f"Total Profit = {total_profit}")

Total Profit = 20.20
